In [96]:
import time 
start_time  = int(time.time())
import pandas as pd 
import numpy as np
from keras.layers import *
from keras.models import Sequential, Model
import pickle

In [97]:
df_train = pd.read_csv("train.csv")
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [98]:
print(len(df_train))

595212


In [99]:
df_train_cat0 = df_train.loc[ df_train['target'] == 0 ]
df_train_cat1 = df_train.loc[ df_train['target'] == 1 ]

In [100]:
df_train = df_train_cat0[:40000].append(df_train_cat1 )

In [101]:
print(df_train['target'].value_counts())

0    40000
1    21694
Name: target, dtype: int64


In [102]:
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [103]:
Y_labels = df_train['target']

In [104]:
df_train = df_train.drop("id", 1)
df_train = df_train.drop("target", 1)
all_cols = df_train.columns

In [105]:
categoical_vars = [ ]
continous_vars = [ ]
categoical_binary_vars = [ ]

for col in all_cols:
    #if "bin" in col :
    #    categoical_binary_vars.append(col)
    if "cat" in col :
        categoical_vars.append(col)
    else:
        continous_vars.append(col)

In [106]:
print ("categorical binary vars: ", len(categoical_binary_vars))
print ("categorical non binary vars: ", len(categoical_vars))
print ("continues vars: ", len(continous_vars))

categorical binary vars:  0
categorical non binary vars:  14
continues vars:  43


In [107]:
for cat_var in categoical_vars:
    print (cat_var, df_train[cat_var].nunique())

ps_ind_02_cat 5
ps_ind_04_cat 3
ps_ind_05_cat 8
ps_car_01_cat 13
ps_car_02_cat 2
ps_car_03_cat 3
ps_car_04_cat 10
ps_car_05_cat 3
ps_car_06_cat 18
ps_car_07_cat 3
ps_car_08_cat 2
ps_car_09_cat 6
ps_car_10_cat 3
ps_car_11_cat 104


In [108]:
df_tr = df_train[:40000]
df_val = df_train[40000:]


Y_train  = Y_labels[:40000]
Y_val  = Y_labels[40000: ]

In [109]:
other_cols = [i for i in all_cols if i not in categoical_vars ]

In [110]:
categoical_vars = categoical_vars[:2]
other_cols = other_cols[:3]

In [111]:
def preproc(X_train ) : 

    input_list_train = []
    
    for c in categoical_vars :
        
        vals = np.asarray(X_train[c].tolist())
        vals = pd.factorize( vals )[0]
        input_list_train.append( np.asarray(vals)  )

    input_list_train.append(X_train[other_cols].values)
    return input_list_train

In [112]:
df_tr_modified = preproc( df_tr )
df_val = preproc(df_val)

In [145]:
outputs = []
inputs = []
for categoical_var in categoical_vars :
    print("------------------------------------------------------------------")
    print("for categoical column ", categoical_var)
    #model = Sequential()
    no_of_unique_cat  = df_train[categoical_var].nunique()
    print("number of unique cat",no_of_unique_cat)
    embedding_size = min(np.ceil((no_of_unique_cat)//2), 50 )
    embedding_size = int(embedding_size)
    print("embedding_size set as ", embedding_size)
    #model.add(  Embedding( no_of_unique_cat+1, embedding_size, input_length = 1 ) )
    
    #model.add(Reshape(target_shape=( [embedding_size] )))
    inp = Input((1, ))
    model = Embedding( no_of_unique_cat+1, embedding_size, input_length = 1 )(inp)
    model = Reshape(target_shape=( [embedding_size] ))(model)
    #print(model.summary())
    
    outputs.append(model)
    inputs.append(inp)

------------------------------------------------------------------
for categoical column  ps_ind_02_cat
number of unique cat 5
embedding_size set as  2
------------------------------------------------------------------
for categoical column  ps_ind_04_cat
number of unique cat 3
embedding_size set as  1


In [146]:
#model_rest = Sequential()
#model_rest.add(Dense(16, input_shape = [3] ))
#model_rest.summary() 
#models.append(model_rest)
inp_rest = Input((3, ))
model_rest = Dense(16, input_shape=[3])(inp_rest)
model_rest = Reshape(target_shape=([16]))(model_rest)
outputs.append(model_rest)
inputs.append(inp_rest)
print(inputs)
print(outputs)
outputs = Concatenate()(outputs)
print(outputs)

[<tf.Tensor 'input_60:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'input_61:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'input_62:0' shape=(?, 3) dtype=float32>]
[<tf.Tensor 'reshape_64/Reshape:0' shape=(?, 2) dtype=float32>, <tf.Tensor 'reshape_65/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'reshape_66/Reshape:0' shape=(?, 16) dtype=float32>]
Tensor("concatenate_21/concat:0", shape=(?, 19), dtype=float32)


In [147]:
outputs = Dense(1024)(outputs)
print(outputs)
outputs = Activation('relu')(outputs)
outputs = Dense(512)(outputs)
print(outputs)
outputs = Activation('relu')(outputs)
outputs = Dense(256)(outputs)
outputs = Activation('sigmoid')(outputs)

outputs = Dense(2)(outputs)
outputs = Activation('sigmoid')(outputs)

full_model = Model(inputs=inputs, outputs=outputs)
full_model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
full_model.summary()

Tensor("dense_120/BiasAdd:0", shape=(?, 1024), dtype=float32)
Tensor("dense_121/BiasAdd:0", shape=(?, 512), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_60 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_61 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_62 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
embedding_38 (Embedding)        (None, 1, 2)         12          input_60[0][0]      

In [148]:
from keras.utils.np_utils import to_categorical
Y_train_cat = to_categorical(Y_train.tolist() )
Y_val_cat = to_categorical(Y_val.tolist()  )

In [150]:
full_model.fit( df_tr_modified, Y_train_cat, epochs=100)

Epoch 1/100
40000/40000 [==============================] - 16s 391us/step - loss: 0.6453 - acc: 0.6486
Epoch 2/100
40000/40000 [==============================] - 15s 383us/step - loss: 0.6445 - acc: 0.6481
Epoch 3/100
40000/40000 [==============================] - 15s 373us/step - loss: 0.6440 - acc: 0.6489
Epoch 4/100
40000/40000 [==============================] - 15s 382us/step - loss: 0.6435 - acc: 0.6488
Epoch 5/100
40000/40000 [==============================] - 16s 391us/step - loss: 0.6432 - acc: 0.6494
Epoch 6/100
40000/40000 [==============================] - 16s 391us/step - loss: 0.6426 - acc: 0.6498
Epoch 7/100
40000/40000 [==============================] - 16s 400us/step - loss: 0.6434 - acc: 0.6495
Epoch 8/100
40000/40000 [==============================] - 16s 393us/step - loss: 0.6429 - acc: 0.6485
Epoch 9/100
40000/40000 [==============================] - 15s 379us/step - loss: 0.6420 - acc: 0.6499
Epoch 10/100
40000/40000 [==============================] - 15s 385us/ste

40000/40000 [==============================] - 15s 375us/step - loss: 0.6370 - acc: 0.6501
Epoch 80/100
40000/40000 [==============================] - 15s 374us/step - loss: 0.6368 - acc: 0.6504
Epoch 81/100
40000/40000 [==============================] - 18s 451us/step - loss: 0.6368 - acc: 0.6507
Epoch 82/100
40000/40000 [==============================] - 18s 441us/step - loss: 0.6367 - acc: 0.6501
Epoch 83/100
40000/40000 [==============================] - 17s 426us/step - loss: 0.6367 - acc: 0.6502
Epoch 84/100
40000/40000 [==============================] - 16s 405us/step - loss: 0.6367 - acc: 0.6501
Epoch 85/100
40000/40000 [==============================] - 17s 422us/step - loss: 0.6365 - acc: 0.6505
Epoch 86/100
40000/40000 [==============================] - 17s 427us/step - loss: 0.6367 - acc: 0.6505
Epoch 87/100
40000/40000 [==============================] - 18s 449us/step - loss: 0.6365 - acc: 0.6506
Epoch 88/100
40000/40000 [==============================] - 16s 394us/step - 